In [1]:
import numpy as np
import zarr
from datetime import datetime

import sys

if not sys.warnoptions:
    import warnings
    warnings.filterwarnings("ignore", category=UserWarning)

In [2]:
store_path = "/eodc/private/tempearth/s1sig0.zarr"

In [3]:
now = datetime.now()
now_np = np.datetime64(now).astype('datetime64[D]')
now_np = np.datetime64("2014-08-12")
origin = np.datetime64("2014-10-01")
timesteps = int((now_np-origin).astype(int))

In [4]:
x_extent = np.arange(4500000, 5400000, 20)
y_extent = np.arange(1800000, 1200000, -20)
time_extent = np.arange(0,timesteps,1)

In [ ]:
shape = (time_extent.shape[0],y_extent.shape[0],x_extent.shape[0])

chunk_shape = (30,60,90)
shard_shape = (30,60,90)

x_shape = x_extent.shape
y_shape = y_extent.shape
time_shape = time_extent.shape

In [ ]:
overwrite=True
store = zarr.storage.LocalStore(store_path)
root = zarr.create_group(store=store, overwrite=overwrite,
                        attributes={"Conventions": "CF-1.7",
                                    "freq": "1D",
                                    "platform": "S5P",
                                    "sensor": "TROPOMI",
                                    "spatial_resolution": 10000,
                                    "collection": "SENTINEL1_SIG0_20M",
                                    "AREA_OR_POINT": "Area",
                                    "esri_pe_string": "PROJCS[\"Azimuthal_Equidistant\",GEOGCS[\"WGS 84\",DATUM[\"WGS_1984\",SPHEROID[\"WGS 84\",6378137,298.257223563,AUTHORITY[\"EPSG\",\"7030\"]],AUTHORITY[\"EPSG\",\"6326\"]],PRIMEM[\"Greenwich\",0],UNIT[\"degree\",0.0174532925199433],AUTHORITY[\"EPSG\",\"4326\"]],PROJECTION[\"Azimuthal_Equidistant\"],PARAMETER[\"latitude_of_center\",53],PARAMETER[\"longitude_of_center\",24],PARAMETER[\"false_easting\",5837287.81977],PARAMETER[\"false_northing\",2121415.69617],UNIT[\"metre\",1,AUTHORITY[\"EPSG\",\"9001\"]]]"})

In [7]:
root.create_array(name="VH",
                    shape=shape,
                    shards=shard_shape,
                    chunks=chunk_shape,
                    compressors = zarr.codecs.BloscCodec(),
                    dtype="int16",
                    fill_value=-9999,
                    dimension_names=["time", "y", "x"],
                    config={"write_empty_chunks":False},
                    attributes={"_FillValue": -9999,
                                "scale_factor": 0.1,
                                "description": "Sigma nought radar backscatter coefficient in VH polarisation",
                                "esri_pe_string": "PROJCS[\"Azimuthal_Equidistant\",GEOGCS[\"WGS 84\",DATUM[\"WGS_1984\",SPHEROID[\"WGS 84\",6378137,298.257223563,AUTHORITY[\"EPSG\",\"7030\"]],AUTHORITY[\"EPSG\",\"6326\"]],PRIMEM[\"Greenwich\",0],UNIT[\"degree\",0.0174532925199433],AUTHORITY[\"EPSG\",\"4326\"]],PROJECTION[\"Azimuthal_Equidistant\"],PARAMETER[\"latitude_of_center\",53],PARAMETER[\"longitude_of_center\",24],PARAMETER[\"false_easting\",5837287.81977],PARAMETER[\"false_northing\",2121415.69617],UNIT[\"metre\",1,AUTHORITY[\"EPSG\",\"9001\"]]]",
                                "long_name": "Sentinel-1 VH polarization sigma nought backscatter",
                                "standard_name": "radar_backscatter_coefficient",
                                "polarisation": "VH",
                                "units": "dB"},
                    overwrite=overwrite)

root.create_array(name="VV",
                    shape=shape,
                    shards=shard_shape,
                    chunks=chunk_shape,
                    compressors = zarr.codecs.BloscCodec(),
                    dtype="int16",
                    fill_value=-9999,
                    dimension_names=["time", "y", "x"],
                    config={"write_empty_chunks":False},
                    attributes={"_FillValue": -9999,
                                "scale_factor": 0.1,
                                "description": "Sigma nought radar backscatter coefficient in VV polarisation",
                                "esri_pe_string": "PROJCS[\"Azimuthal_Equidistant\",GEOGCS[\"WGS 84\",DATUM[\"WGS_1984\",SPHEROID[\"WGS 84\",6378137,298.257223563,AUTHORITY[\"EPSG\",\"7030\"]],AUTHORITY[\"EPSG\",\"6326\"]],PRIMEM[\"Greenwich\",0],UNIT[\"degree\",0.0174532925199433],AUTHORITY[\"EPSG\",\"4326\"]],PROJECTION[\"Azimuthal_Equidistant\"],PARAMETER[\"latitude_of_center\",53],PARAMETER[\"longitude_of_center\",24],PARAMETER[\"false_easting\",5837287.81977],PARAMETER[\"false_northing\",2121415.69617],UNIT[\"metre\",1,AUTHORITY[\"EPSG\",\"9001\"]]]",
                                "long_name": "Sentinel-1 VV polarization sigma nought backscatter",
                                "standard_name": "radar_backscatter_coefficient",
                                "polarisation": "VV",
                                "units": "dB"},
                    overwrite=overwrite)

root.create_array(name="sensing_date",
                shape=shape,
                shards=(30,5000,5000),
                chunks=(30,250,250),
                dtype="int64",
                fill_value=-9999,
                dimension_names=["time", "y", "x"],
                attributes={"units": "seconds since 2014-10-01 00:00:00",
                            "calendar": "proleptic_gregorian",
                            "_FillValue": -9999,
                            "esri_pe_string": "PROJCS[\"Azimuthal_Equidistant\",GEOGCS[\"WGS 84\",DATUM[\"WGS_1984\",SPHEROID[\"WGS 84\",6378137,298.257223563,AUTHORITY[\"EPSG\",\"7030\"]],AUTHORITY[\"EPSG\",\"6326\"]],PRIMEM[\"Greenwich\",0],UNIT[\"degree\",0.0174532925199433],AUTHORITY[\"EPSG\",\"4326\"]],PROJECTION[\"Azimuthal_Equidistant\"],PARAMETER[\"latitude_of_center\",53],PARAMETER[\"longitude_of_center\",24],PARAMETER[\"false_easting\",5837287.81977],PARAMETER[\"false_northing\",2121415.69617],UNIT[\"metre\",1,AUTHORITY[\"EPSG\",\"9001\"]]]",
                            "long_name": "sensing time of measurement",
                            "standard_name": "time"},
                overwrite=overwrite)

root.create_array(name="absolute_orbit_number",
                shape=shape,
                shards=(30,5000,5000),
                chunks=(30,250,250),
                dtype="int32",
                fill_value=-9999,
                dimension_names=["time", "y", "x"],
                attributes={"_FillValue": -9999,
                            "esri_pe_string": "PROJCS[\"Azimuthal_Equidistant\",GEOGCS[\"WGS 84\",DATUM[\"WGS_1984\",SPHEROID[\"WGS 84\",6378137,298.257223563,AUTHORITY[\"EPSG\",\"7030\"]],AUTHORITY[\"EPSG\",\"6326\"]],PRIMEM[\"Greenwich\",0],UNIT[\"degree\",0.0174532925199433],AUTHORITY[\"EPSG\",\"4326\"]],PROJECTION[\"Azimuthal_Equidistant\"],PARAMETER[\"latitude_of_center\",53],PARAMETER[\"longitude_of_center\",24],PARAMETER[\"false_easting\",5837287.81977],PARAMETER[\"false_northing\",2121415.69617],UNIT[\"metre\",1,AUTHORITY[\"EPSG\",\"9001\"]]]",
                            "long_name": "Absolute orbit number",
                            "units": ""},
                overwrite=overwrite)

root.create_array(name="relative_orbit_number",
                shape=shape,
                shards=(30,5000,5000),
                chunks=(30,250,250),
                dtype="int16",
                fill_value=-9999,
                dimension_names=["time", "y", "x"],
                attributes={"_FillValue": -9999,
                            "esri_pe_string": "PROJCS[\"Azimuthal_Equidistant\",GEOGCS[\"WGS 84\",DATUM[\"WGS_1984\",SPHEROID[\"WGS 84\",6378137,298.257223563,AUTHORITY[\"EPSG\",\"7030\"]],AUTHORITY[\"EPSG\",\"6326\"]],PRIMEM[\"Greenwich\",0],UNIT[\"degree\",0.0174532925199433],AUTHORITY[\"EPSG\",\"4326\"]],PROJECTION[\"Azimuthal_Equidistant\"],PARAMETER[\"latitude_of_center\",53],PARAMETER[\"longitude_of_center\",24],PARAMETER[\"false_easting\",5837287.81977],PARAMETER[\"false_northing\",2121415.69617],UNIT[\"metre\",1,AUTHORITY[\"EPSG\",\"9001\"]]]",
                            "long_name": "Relative orbit number",
                            "units": ""},                
                overwrite=overwrite)

x_array = root.create_array(name="x",
            shape=x_shape,
            chunks=x_shape,
            dtype="float64",
            dimension_names=["x"],
            attributes={"_FillValue": "AAAAAAAA+H8=", #fill value is NaN
                        "axis": "X",
                        "long_name": "x coordinate of projection",
                        "standard_name": "projection_x_coordinate",
                        "units": "m"},
            overwrite=overwrite)

y_array = root.create_array(name="y",
            shape=y_shape,
            chunks=y_shape,
            dtype="float64",
            dimension_names=["y"],
            attributes={"_FillValue": "AAAAAAAA+H8=", #fill value is NaN
                        "axis": "Y",
                        "long_name": "y coordinate of projection",
                        "standard_name": "projection_y_coordinate",
                        "units": "m"},
            overwrite=overwrite)

time_array = root.create_array(name="time",
            shape=time_shape,
            dtype="int64",
            dimension_names=["time"],
            attributes={"units": "days since 2014-10-01",
                        "calendar": "proleptic_gregorian",
                        "long_name": "date of acquisition",
                        "standard_name": "time"},
            overwrite=overwrite)

In [8]:
x_array[:] = x_extent
y_array[:] = y_extent
time_array[:] = time_extent
zarr.consolidate_metadata(store)

<Group file:///eodc/private/tempearth/s1sig0.zarr>

In [15]:
now = datetime.now()
now_np = np.datetime64(now).astype('datetime64[D]')
origin = np.datetime64("2014-10-01")

new_shape = int((now_np-origin).astype(int))
new_extent = np.arange(0,new_shape,1)

store = zarr.storage.LocalStore(store_path)
group = zarr.group(store=store)

array_names=set(group.array_keys())
coords = {"time", "x", "y"}
data_arrays = array_names-coords

group["time"].resize(new_shape)
for array in data_arrays:
    group_shape  = group[array].shape
    group[array].resize((new_shape, group_shape[1], group_shape[2]))

zarr.consolidate_metadata(store)

store = zarr.storage.LocalStore(store_path)
group = zarr.group(store=store)

group["time"][:]=new_extent